<a href="https://colab.research.google.com/github/adammoss/MLiS2/blob/reinforcement_learning/examples/reinforcement_learning/tabular/monte_carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We begin by defining our environment as a class: we need this to have two methods which the agent can interact with, to induce transitions in the state, and to restart the environment after each episode, which we call transition and reset respectively. Transition calls _reward, which provides the reward for the transition which just occured, and provided the new state and reward as output to the agent.